# Contrastive Learning From Scratch - DistilBERT

An attempt to build contrastive learning model from scratch. Parts include:

- Loading and preparing Wiki-1M data for model input
- Contrastive learning model
  - Forward passing using pre-trained model
  - Constrastive layer
  - Calculate loss
- Training procedure
  - Default trainer optimizer
  - Default trainer hyper-parameters

In [1]:
import os

# Set Project home
PROJECT_HOME = os.path.join('/',
                            'Users',
                            'ng-ka',
                            'OMSCS',
                            'DL',
                            'DLProject',
                            'contrastive-learning-in-distilled-models')
%cd {PROJECT_HOME}

# Load project code
%reload_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, './src')

#import distilface
import src.distilface as distilface

C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models


In [2]:
os.getcwd()

'C:\\Users\\ng-ka\\OMSCS\\DL\\DLProject\\contrastive-learning-in-distilled-models'

In [28]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer, DistilBertModel, DistilBertPreTrainedModel, AutoConfig
from transformers.modeling_outputs import SequenceClassifierOutput, BaseModelOutputWithPooling

from src.distilface.modules.pooler import Pooler
from src.distilface.modules.similarity import Similarity

class DistilBertCLModel(DistilBertPreTrainedModel):
    def __init__(self, config, pooler_type='avg_last4', temp=0.05):
        super().__init__(config)

        self.config = config
        self.pooler_type = pooler_type
        self.temp = 0.05

        self.distilbert = DistilBertModel(config)
        self.pooler = Pooler(pooler_type)
        self.sim = Similarity(temp=temp)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None):
        if self.training:
            return self.cl_forward(self.distilbert, input_ids, attention_mask)
        else:
            return self.sent_emb(self.distilbert, input_ids, attention_mask)

    def cl_forward(self, encoder, input_ids=None, attention_mask=None):
        batch_size = input_ids.size(0)#64#input_ids.size(0)
        num_sent = input_ids.size(1)  # Number of sentences in one instance: 2 sentences

        # Flatten all input tensors
        input_ids = input_ids.view((-1, input_ids.size(-1))) # (bs * num_sent, len)
        attention_mask = attention_mask.view((-1, attention_mask.size(-1))) # (bs * num_sent len)

        # Pre-trained Model Encoder
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )

        # Pooling
        pooler_output = self.pooler(attention_mask, outputs)
        pooler_output = pooler_output.view((batch_size, num_sent, pooler_output.size(-1)))  # (bs, num_sent, hidden)

        # Separate representation
        z1, z2 = pooler_output[:, 0], pooler_output[:, 1]

        # Cosine similarity
        cos_sim = self.sim(z1.unsqueeze(1), z2.unsqueeze(0))

        # Calculate contrastive loss
        criterion = nn.CrossEntropyLoss()
        labels = torch.arange(cos_sim.size(0)).long().to(self.device)
        loss = criterion(cos_sim, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=cos_sim,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def sent_emb(self, encoder, input_ids=None, attention_mask=None):
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )
        pooler_output = self.pooler(attention_mask, outputs)

        return BaseModelOutputWithPooling(
            pooler_output=pooler_output,
            last_hidden_state=outputs.last_hidden_state,
            hidden_states=outputs.hidden_states,
        )


pretrained_model_name = 'distilbert-base-uncased'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(pretrained_model_name)

model = DistilBertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

#model.eval();


loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at C:\Users\ng-ka/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.17.0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at C:\Users\ng-ka/.cache\huggingface\transfor

In [29]:
model = torch.load('batch128_model.pth')
model.eval()

DistilBertCLModel(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linea

In [30]:
model = model.to('cpu')

In [31]:
quantized_model =torch.quantization.quantize_dynamic(model, qconfig_spec={nn.Linear},  dtype=torch.qint8
)

In [32]:
def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (KB):', size/1e3)
    os.remove('temp.p')
    return size

# compare the sizes
f=print_size_of_model(model,"fp32")
q=print_size_of_model(quantized_model,"int8")
print("{0:.2f} times smaller".format(f/q))

model:  fp32  	 Size (KB): 265489.337
model:  int8  	 Size (KB): 138116.329
1.92 times smaller


The following does not succeed due to lack of compatibility of our model with Torch Script

In [27]:
scripted_model = torch.jit.script(quantized_model)

UnsupportedNodeError: GeneratorExp aren't supported:
  File "C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\transformers\modeling_utils.py", line 999
    
        x=False
        for it_element in (hasattr(m, "gradient_checkpointing") and m.gradient_checkpointing for m in self.modules()):
                          ~ <--- HERE
            if it_element:
                x = True


Attempts with the alternative method of tracing were also not successful

In [36]:
from transformers import BertModel, BertTokenizer, BertConfig

enc = BertTokenizer.from_pretrained("distilbert-base-uncased")
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = enc.tokenize(text)
indexed_tokens = enc.convert_tokens_to_ids(tokenized_text)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


# Creating a dummy input
tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')
segments_tensors = torch.tensor([segments_ids]).to('cuda')

traced_model = torch.jit.trace(quantized_model, [tokens_tensor, segments_tensors])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


RuntimeError: Tracer cannot infer type of BaseModelOutputWithPooling(last_hidden_state=tensor([[[-2.8095e-01,  1.9375e-01, -2.3601e-01,  ...,  2.4750e-04,
           1.8413e-01,  7.4281e-01],
         [-2.0558e-01,  5.6973e-01, -3.4450e-01,  ...,  3.1061e-01,
           7.4593e-03,  6.6465e-01],
         [-2.6471e-01,  4.7577e-01, -8.7475e-01,  ...,  4.3364e-01,
           1.6303e-02,  1.2238e+00],
         ...,
         [ 2.1979e-01,  2.8544e-02, -4.8508e-01,  ...,  1.5100e-01,
           5.6132e-01,  6.3370e-01],
         [ 8.1528e-02,  5.9031e-01, -1.4732e-01,  ..., -5.9024e-02,
           5.9021e-01,  8.1966e-01],
         [-7.2517e-03,  1.6594e-01, -7.6334e-01,  ...,  5.0187e-01,
           3.8227e-01,  5.1416e-01]]], device='cuda:0',
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 4.6693e-02,  4.0107e-02, -3.7071e-01, -2.7972e-02, -9.5654e-02,
         -8.7964e-02,  1.8528e-02,  2.7150e-01, -2.4230e-01,  9.8927e-05,
         -3.4853e-01, -5.7866e-01, -1.3977e-

In [38]:
opt_model = optimize_for_mobile(traced_model)

RuntimeError: Tracer cannot infer type of BaseModelOutputWithPooling(last_hidden_state=tensor([[[-2.8095e-01,  1.9375e-01, -2.3601e-01,  ...,  2.4750e-04,
           1.8413e-01,  7.4281e-01],
         [-2.0558e-01,  5.6973e-01, -3.4450e-01,  ...,  3.1061e-01,
           7.4593e-03,  6.6465e-01],
         [-2.6471e-01,  4.7577e-01, -8.7475e-01,  ...,  4.3364e-01,
           1.6303e-02,  1.2238e+00],
         ...,
         [ 2.1979e-01,  2.8544e-02, -4.8508e-01,  ...,  1.5100e-01,
           5.6132e-01,  6.3370e-01],
         [ 8.1528e-02,  5.9031e-01, -1.4732e-01,  ..., -5.9024e-02,
           5.9021e-01,  8.1966e-01],
         [-7.2517e-03,  1.6594e-01, -7.6334e-01,  ...,  5.0187e-01,
           3.8227e-01,  5.1416e-01]]], device='cuda:0',
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 4.6693e-02,  4.0107e-02, -3.7071e-01, -2.7972e-02, -9.5654e-02,
         -8.7964e-02,  1.8528e-02,  2.7150e-01, -2.4230e-01,  9.8927e-05,
         -3.4853e-01, -5.7866e-01, -1.3977e-

In [3]:
# import torch
# torch.save(quantized_model, './quantized_model.pth')

C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'quantized_model' is not defined

Compare with default BERT

## 1. Loading and Preparing Wiki-1M data

Use huggingface `datasets` library to load local file data.

In [4]:
import numpy as np

from datasets import load_dataset

data_files = {'train': 'data/training/wiki1m_for_simcse.txt'}
datasets = load_dataset('text', data_files=data_files)

Using custom data configuration default-84caea1147087fa9
Reusing dataset text (C:\Users\ng-ka\.cache\huggingface\datasets\text\default-84caea1147087fa9\0.0.0\4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.44it/s]


In [5]:
# Unsupervised / Self-supervised dataset

column_names = datasets["train"].column_names
sent0_cname = column_names[0]
sent1_cname = column_names[0]

print('column_names:', column_names)
print('sent0_cname:', sent0_cname, '| sent1_cname:', sent1_cname)

column_names: ['text']
sent0_cname: text | sent1_cname: text


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [7]:
def prepare_features(examples):
    total = len(examples[sent0_cname])

    # Avoid "None" fields 
    for idx in range(total):
        if examples[sent0_cname][idx] is None:
            examples[sent0_cname][idx] = " "
        if examples[sent1_cname][idx] is None:
            examples[sent1_cname][idx] = " "
    
    sentences = examples[sent0_cname] + examples[sent1_cname]

    sent_features = tokenizer(
        sentences,
        max_length=32,
        truncation=True,
        padding=True,
    )

    features = {}
    for key in sent_features:
        features[key] = [[sent_features[key][i], sent_features[key][i+total]] for i in range(total)]

    return features

In [8]:
train_dataset = datasets["train"].map(prepare_features,
                                      batched=True,
                                    #   num_proc=24,
                                      remove_columns=column_names)

Loading cached processed dataset at C:\Users\ng-ka\.cache\huggingface\datasets\text\default-84caea1147087fa9\0.0.0\4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8\cache-dd04d9dec26aeca9.arrow


In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased")


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Sentence 1 and Sentence 2 are the same sentence

## 2. Contrastive Learning Model

In [9]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer, BertModel, BertPreTrainedModel, AutoConfig
from transformers.modeling_outputs import SequenceClassifierOutput, BaseModelOutputWithPooling

from distilface.modules.pooler import Pooler
from distilface.modules.similarity import Similarity


class BertCLModel(BertPreTrainedModel):
    def __init__(self, config, pooler_type='avg_first_last', temp=0.05):
        super().__init__(config)

        self.config = config
        self.pooler_type = pooler_type
        self.temp = 0.05

        self.bert = BertModel(config, add_pooling_layer=False)
        self.pooler = Pooler(pooler_type)
        self.sim = Similarity(temp=temp)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
        if self.training:
            return self.cl_forward(self.bert, input_ids, attention_mask, token_type_ids)
        else:
            return self.sent_emb(self.bert, input_ids, attention_mask, token_type_ids)

    def cl_forward(self, encoder, input_ids=None, attention_mask=None, token_type_ids=None):
        batch_size = input_ids.size(0)
        num_sent = input_ids.size(1)  # Number of sentences in one instance: 2 sentences

        # Flatten all input tensors
        input_ids = input_ids.view((-1, input_ids.size(-1))) # (bs * num_sent, len)
        attention_mask = attention_mask.view((-1, attention_mask.size(-1))) # (bs * num_sent len)
        token_type_ids = token_type_ids.view((-1, token_type_ids.size(-1))) # (bs * num_sent, len)

        # Pre-trained Model Encoder
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_attentions=True,
            output_hidden_states=True,
            return_dict=True,
        )

        # Pooling
        pooler_output = self.pooler(attention_mask, outputs)
        pooler_output = pooler_output.view((batch_size, num_sent, pooler_output.size(-1)))  # (bs, num_sent, hidden)

        # Separate representation
        z1, z2 = pooler_output[:, 0], pooler_output[:, 1]

        # Cosine similarity
        cos_sim = self.sim(z1.unsqueeze(1), z2.unsqueeze(0))

        # Calculate contrastive loss
        criterion = nn.CrossEntropyLoss()
        labels = torch.arange(cos_sim.size(0)).long().to(self.device)
        loss = criterion(cos_sim, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=cos_sim,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def sent_emb(self, encoder, input_ids=None, attention_mask=None, token_type_ids=None):
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_attentions=True,
            output_hidden_states=True,
            return_dict=True,
        )
        pooler_output = self.pooler(attention_mask, outputs)

        return BaseModelOutputWithPooling(
            pooler_output=pooler_output,
            last_hidden_state=outputs.last_hidden_state,
            hidden_states=outputs.hidden_states,
        )


pretrained_model_name = 'bert-base-uncased'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(pretrained_model_name)

model = BertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

model.eval();


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertCLModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertCLModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertCLModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 2.1 Initial BERT embeddings performance

In [10]:
import senteval

def prepare(params, samples):
    return

def batcher(params, batch):
    sentences = [" ".join(s) for s in batch]
    batch = tokenizer.batch_encode_plus(
        sentences,
        return_tensors="pt",
        padding=True,
    )

    for k in batch:
        batch[k] = batch[k].to(device)

    with torch.no_grad():
        outputs = model(**batch)

    pooled_result = outputs.pooler_output.cpu()

    return pooled_result


def evaluate_model():
    PATH_TO_DATA = "./data"

    params = {"task_path": PATH_TO_DATA, "usepytorch": True, "kfold": 10}
    tasks = ["STSBenchmark", 'STS12', 'STS13', 'STS14', 'STS15']

    se = senteval.engine.SE(params, batcher, prepare)
    #se = se_engine.SE(params, batcher, prepare)
    results = se.eval(tasks)

    print('STS12: ', results["STS12"]["all"]["spearman"]["all"])
    print('STS13: ', results["STS13"]["all"]["spearman"]["all"])
    print('STS14: ', results["STS14"]["all"]["spearman"]["all"])
    print('STS15: ', results["STS15"]["all"]["spearman"]["all"])
    print('STSB: ', results["STSBenchmark"]["test"]["spearman"][0])

    return results

## 3. Trainer

In [15]:
import mlflow

from transformers import Trainer, TrainingArguments
from transformers import default_data_collator

training_args = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=True,
    learning_rate=1e-05,
    per_device_train_batch_size= 128,
    per_device_eval_batch_size = 128,
    weight_decay=0.0,
    num_train_epochs=2,
    max_steps= 30000,
    logging_steps=5000,
    save_steps=5000
)

model.train()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
max_steps is given, it will override any value given in num_train_epochs


In [21]:
os.getcwd()

'C:\\Users\\ng-ka\\OMSCS\\DL\\DLProject\\contrastive-learning-in-distilled-models'

In [ ]:
model_path = 'trained_model/bert_cl'

train_result = trainer.train()
torch.save(model, './bert_model_best_params.pth')

C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000000
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss
5000,0.000200


Saving model checkpoint to output\checkpoint-5000
Configuration saved in output\checkpoint-5000\config.json
Model weights saved in output\checkpoint-5000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-5000\tokenizer_config.json
Special tokens file saved in output\checkpoint-5000\special_tokens_map.json


In [14]:
def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (KB):', size/1e3)
    os.remove('temp.p')
    return size

# compare the sizes
f=print_size_of_model(model,"fp32")

model:  fp32  	 Size (KB): 435650.293


In [ ]:
q=print_size_of_model(quantized_model,"int8")
print("{0:.2f} times smaller".format(f/q))

In [37]:
mlflow.end_run()

## 4. Evaluate DistilBert CL Model performance

In [16]:
model.eval()

results = evaluate_model()
results

C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


STS12:  0.6522348679699254
STS13:  0.7245264269994489
STS14:  0.6591385536542372
STS15:  0.7803083813296277
STSB:  0.7317242200449551


{'STSBenchmark': {'train': {'pearson': (0.7701978269277896, 0.0),
   'spearman': SpearmanrResult(correlation=0.7444988754489594, pvalue=0.0),
   'nsamples': 5749},
  'dev': {'pearson': (0.792601430214866, 0.0),
   'spearman': SpearmanrResult(correlation=0.7923150663580373, pvalue=0.0),
   'nsamples': 1500},
  'test': {'pearson': (0.7395838046909169, 4.979487826512024e-239),
   'spearman': SpearmanrResult(correlation=0.7317242200449551, pvalue=1.7326856147338482e-231),
   'nsamples': 1379},
  'all': {'pearson': {'all': 0.7683647566030786,
    'mean': 0.7674610206111909,
    'wmean': 0.7691997588084071},
   'spearman': {'all': 0.7548919636159135,
    'mean': 0.7561793872839839,
    'wmean': 0.7507700897004076}}},
 'STS12': {'MSRpar': {'pearson': (0.6587949262407169, 1.5689202234706753e-94),
   'spearman': SpearmanrResult(correlation=0.6366967124028455, pvalue=1.678438485809478e-86),
   'nsamples': 750},
  'MSRvid': {'pearson': (0.8536988445568228, 3.8250578501088855e-214),
   'spearman':

# 64 batch size

In [21]:
mlflow.end_run()

In [18]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer, DistilBertModel, DistilBertPreTrainedModel, AutoConfig
from transformers.modeling_outputs import SequenceClassifierOutput, BaseModelOutputWithPooling

from src.distilface.modules.pooler import Pooler
from src.distilface.modules.similarity import Similarity

from torch.cuda.amp import autocast 

#scaler = GradScaler()

class DistilBertCLModel(DistilBertPreTrainedModel):
    def __init__(self, config, pooler_type='avg_first_last', temp=0.05):
        super().__init__(config)

        self.config = config
        self.pooler_type = pooler_type
        self.temp = 0.05

        self.distilbert = DistilBertModel(config)
        self.pooler = Pooler(pooler_type)
        self.sim = Similarity(temp=temp)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None):
        with autocast():
            if self.training:
                return self.cl_forward(self.distilbert, input_ids, attention_mask)
            else:
                return self.sent_emb(self.distilbert, input_ids, attention_mask)

    def cl_forward(self, encoder, input_ids=None, attention_mask=None):
        batch_size = input_ids.size(0)#64#input_ids.size(0)
        num_sent = input_ids.size(1)  # Number of sentences in one instance: 2 sentences

        # Flatten all input tensors
        input_ids = input_ids.view((-1, input_ids.size(-1))) # (bs * num_sent, len)
        attention_mask = attention_mask.view((-1, attention_mask.size(-1))) # (bs * num_sent len)

        # Pre-trained Model Encoder
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )

        # Pooling
        pooler_output = self.pooler(attention_mask, outputs)
        pooler_output = pooler_output.view((batch_size, num_sent, pooler_output.size(-1)))  # (bs, num_sent, hidden)

        # Separate representation
        z1, z2 = pooler_output[:, 0], pooler_output[:, 1]

        # Cosine similarity
        cos_sim = self.sim(z1.unsqueeze(1), z2.unsqueeze(0))

        # Calculate contrastive loss
        criterion = nn.CrossEntropyLoss()
        labels = torch.arange(cos_sim.size(0)).long().to(self.device)
        loss = criterion(cos_sim, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=cos_sim,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def sent_emb(self, encoder, input_ids=None, attention_mask=None):
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )
        pooler_output = self.pooler(attention_mask, outputs)

        return BaseModelOutputWithPooling(
            pooler_output=pooler_output,
            last_hidden_state=outputs.last_hidden_state,
            hidden_states=outputs.hidden_states,
        )


pretrained_model_name = 'distilbert-base-uncased'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(pretrained_model_name)

model2 = DistilBertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

#model.eval();


loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at C:\Users\ng-ka/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.17.0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at C:\Users\ng-ka/.cache\huggingface\transfor

In [19]:
#model2 = DistilBertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)

training_args2 = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=False,
    learning_rate=5e-05,
    per_device_train_batch_size= 64,
    per_device_eval_batch_size = 64,
    weight_decay=0.0,
    num_train_epochs=2,
    max_steps= 30000,
    logging_steps=5000,
    save_steps=5000,
    fp16=True
)

model2.train()

trainer2 = Trainer(
    model=model2,
    args=training_args2,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
max_steps is given, it will override any value given in num_train_epochs
Using amp half precision backend


In [22]:
model_path = 'trained_model/distilbert_cl'

#train_result = trainer.train(model_path=model_path)
train_result2 = trainer2.train()
torch.save(model2, './batch64_fp16_model.pth')

***** Running training *****
  Num examples = 1000000
  Num Epochs = 2
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss
5000,0.000100
10000,0.000100
15000,0.000100
20000,0.000100
25000,0.000100
30000,0.000100


Saving model checkpoint to output\checkpoint-5000
Configuration saved in output\checkpoint-5000\config.json
Model weights saved in output\checkpoint-5000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-5000\tokenizer_config.json
Special tokens file saved in output\checkpoint-5000\special_tokens_map.json
Saving model checkpoint to output\checkpoint-10000
Configuration saved in output\checkpoint-10000\config.json
Model weights saved in output\checkpoint-10000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-10000\tokenizer_config.json
Special tokens file saved in output\checkpoint-10000\special_tokens_map.json
Saving model checkpoint to output\checkpoint-15000
Configuration saved in output\checkpoint-15000\config.json
Model weights saved in output\checkpoint-15000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-15000\tokenizer_config.json
Special tokens file saved in output\checkpoint-15000\special_tokens_map.json
Saving model check

In [23]:
import senteval
#import SentEval.senteval as senteval
#import SentEval_simcse.senteval as senteval
#import SentEval_simcse.senteval.engine as se_engine


def prepare(params, samples):
    return

def batcher(params, batch):
    sentences = [" ".join(s) for s in batch]
    batch = tokenizer.batch_encode_plus(
        sentences,
        return_tensors="pt",
        padding=True,
    )

    for k in batch:
        batch[k] = batch[k].to(device)

    with torch.no_grad():
        outputs = model2(**batch)

    pooled_result = outputs.pooler_output.cpu()

    return pooled_result


def evaluate_model():
    PATH_TO_DATA = "./data"

    params = {"task_path": PATH_TO_DATA, "usepytorch": True, "kfold": 10}
    tasks = ["STSBenchmark", 'STS12', 'STS13', 'STS14', 'STS15']

    se = senteval.engine.SE(params, batcher, prepare)
    #se = se_engine.SE(params, batcher, prepare)
    results = se.eval(tasks)

    print('STS12: ', results["STS12"]["all"]["spearman"]["all"])
    print('STS13: ', results["STS13"]["all"]["spearman"]["all"])
    print('STS14: ', results["STS14"]["all"]["spearman"]["all"])
    print('STS15: ', results["STS15"]["all"]["spearman"]["all"])
    print('STSB: ', results["STSBenchmark"]["test"]["spearman"][0])

    return results

In [24]:
model2.eval()

results2 = evaluate_model()
results2

C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


STS12:  0.5860086657732572
STS13:  0.7462740979049128
STS14:  0.6726454938423136
STS15:  0.7632500735789172
STSB:  0.7239425725655111


{'STSBenchmark': {'train': {'pearson': (0.7514974091222588, 0.0),
   'spearman': SpearmanrResult(correlation=0.728916173975978, pvalue=0.0),
   'nsamples': 5749},
  'dev': {'pearson': (0.7515928663596871, 5.587693491694605e-273),
   'spearman': SpearmanrResult(correlation=0.7571062733767996, pvalue=2.91924659407765e-279),
   'nsamples': 1500},
  'test': {'pearson': (0.7324146663602896, 3.863341707867918e-232),
   'spearman': SpearmanrResult(correlation=0.7239425725655111, pvalue=2.7955081361595227e-224),
   'nsamples': 1379},
  'all': {'pearson': {'all': 0.745320014694186,
    'mean': 0.7451683139474118,
    'wmean': 0.7484640391161609},
   'spearman': {'all': 0.7376444193730962,
    'mean': 0.7366550066394296,
    'wmean': 0.7330221722091952}}},
 'STS12': {'MSRpar': {'pearson': (0.6420318665513999, 2.2231580917506402e-88),
   'spearman': SpearmanrResult(correlation=0.6250375288907182, pvalue=1.5936298376149268e-82),
   'nsamples': 750},
  'MSRvid': {'pearson': (0.7821232084995746, 8.1

In [31]:
mlflow.end_run()

# 256 batch size

In [10]:
import mlflow

from transformers import Trainer, TrainingArguments
from transformers import default_data_collator

model3 = DistilBertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)

training_args3 = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=True,
    learning_rate=5e-05,
    per_device_train_batch_size= 256,
    per_device_eval_batch_size = 256,
    weight_decay=0.0,
    num_train_epochs=2,
    max_steps= 30000,
    logging_steps=10000,
    save_steps=10000
)

model3.train()

trainer3 = Trainer(
    model=model3,
    args=training_args3,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertCLModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertCLModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertCLModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
max_steps is given, it will override any value given in num_train_epochs


In [11]:
model_path = 'trained_model/distilbert_cl'

#train_result = trainer.train(model_path=model_path)
train_result3 = trainer3.train()
torch.save(model3, './batch256_model.pth')

C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000000
  Num Epochs = 8
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss
10000,0.000500
20000,0.000400
30000,0.000300


Saving model checkpoint to output\checkpoint-10000
Configuration saved in output\checkpoint-10000\config.json
Model weights saved in output\checkpoint-10000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-10000\tokenizer_config.json
Special tokens file saved in output\checkpoint-10000\special_tokens_map.json
Saving model checkpoint to output\checkpoint-20000
Configuration saved in output\checkpoint-20000\config.json
Model weights saved in output\checkpoint-20000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-20000\tokenizer_config.json
Special tokens file saved in output\checkpoint-20000\special_tokens_map.json
Saving model checkpoint to output\checkpoint-30000
Configuration saved in output\checkpoint-30000\config.json
Model weights saved in output\checkpoint-30000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-30000\tokenizer_config.json
Special tokens file saved in output\checkpoint-30000\special_tokens_map.json


Training co

In [26]:
#mlflow.end_run()

In [12]:
model3.eval()

results3 = evaluate_model()
results3

C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


STS12:  0.5697409578579856
STS13:  0.6769343320004665
STS14:  0.6357083664225791
STS15:  0.744575188106335
STSB:  0.6320978112448171


{'STSBenchmark': {'train': {'pearson': (0.7108974767715875, 0.0),
   'spearman': SpearmanrResult(correlation=0.6957934629797473, pvalue=0.0),
   'nsamples': 5749},
  'dev': {'pearson': (0.7092411378218643, 1.0445253840454627e-229),
   'spearman': SpearmanrResult(correlation=0.7145470998781457, pvalue=1.0790202728598763e-234),
   'nsamples': 1500},
  'test': {'pearson': (0.6362564654216417, 2.357894834089188e-157),
   'spearman': SpearmanrResult(correlation=0.6320978112448171, pvalue=1.0318637044184543e-154),
   'nsamples': 1379},
  'all': {'pearson': {'all': 0.694542147265105,
    'mean': 0.6854650266716978,
    'wmean': 0.6986797596788475},
   'spearman': {'all': 0.692993221621729,
    'mean': 0.68081279136757,
    'wmean': 0.6888734527346301}}},
 'STS12': {'MSRpar': {'pearson': (0.5685692938261935, 1.922991733409069e-65),
   'spearman': SpearmanrResult(correlation=0.5768180019887325, pvalue=9.865649661539039e-68),
   'nsamples': 750},
  'MSRvid': {'pearson': (0.7231459307017963, 2.46

In [17]:
mlflow.end_run()

In [11]:
class DistilBertCLModel(DistilBertPreTrainedModel):
    def __init__(self, config, pooler_type='avg_first_last', temp=0.05):
        super().__init__(config)

        self.config = config
        self.pooler_type = pooler_type
        self.temp = 0.05

        self.distilbert = DistilBertModel(config)
        self.pooler = Pooler(pooler_type)
        self.sim = Similarity(temp=temp)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None):
        if self.training:
            return self.cl_forward(self.distilbert, input_ids, attention_mask)
        else:
            return self.sent_emb(self.distilbert, input_ids, attention_mask)

    def cl_forward(self, encoder, input_ids=None, attention_mask=None):
        batch_size = input_ids.size(0)#64#input_ids.size(0)
        num_sent = input_ids.size(1)  # Number of sentences in one instance: 2 sentences

        # Flatten all input tensors
        input_ids = input_ids.view((-1, input_ids.size(-1))) # (bs * num_sent, len)
        attention_mask = attention_mask.view((-1, attention_mask.size(-1))) # (bs * num_sent len)

        # Pre-trained Model Encoder
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )

        # Pooling
        pooler_output = self.pooler(attention_mask, outputs)
        pooler_output = pooler_output.view((batch_size, num_sent, pooler_output.size(-1)))  # (bs, num_sent, hidden)

        # Separate representation
        z1, z2 = pooler_output[:, 0], pooler_output[:, 1]

        # Cosine similarity
        cos_sim = self.sim(z1.unsqueeze(1), z2.unsqueeze(0))

        # Calculate contrastive loss
        criterion = nn.CrossEntropyLoss()
        labels = torch.arange(cos_sim.size(0)).long().to(self.device)
        loss = criterion(cos_sim, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=cos_sim,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def sent_emb(self, encoder, input_ids=None, attention_mask=None):
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )
        pooler_output = self.pooler(attention_mask, outputs)

        return BaseModelOutputWithPooling(
            pooler_output=pooler_output,
            last_hidden_state=outputs.last_hidden_state,
            hidden_states=outputs.hidden_states,
        )

In [15]:
import mlflow

from transformers import Trainer, TrainingArguments
from transformers import default_data_collator

model4 = DistilBertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)

training_args4 = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=True,
    learning_rate=5e-05,
    per_device_train_batch_size= 256,
    per_device_eval_batch_size = 256,
    weight_decay=0.0,
    num_train_epochs=2,
    max_steps= 30000,
    logging_steps=10000,
    save_steps=10000,
    fp16=True
)

model4.train()

trainer4 = Trainer(
    model=model4,
    args=training_args4,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertCLModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertCLModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertCLModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
max_steps is given, it will override any value given in num_train_epochs
Using amp half precision backend


In [16]:
model_path = 'trained_model/distilbert_cl'

#train_result = trainer.train(model_path=model_path)
train_result4 = trainer4.train()
torch.save(model4, './batch256_16bit_model.pth')

C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000000
  Num Epochs = 8
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss
10000,0.000400
20000,0.000300
30000,0.000300


Saving model checkpoint to output\checkpoint-10000
Configuration saved in output\checkpoint-10000\config.json
Model weights saved in output\checkpoint-10000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-10000\tokenizer_config.json
Special tokens file saved in output\checkpoint-10000\special_tokens_map.json
Saving model checkpoint to output\checkpoint-20000
Configuration saved in output\checkpoint-20000\config.json
Model weights saved in output\checkpoint-20000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-20000\tokenizer_config.json
Special tokens file saved in output\checkpoint-20000\special_tokens_map.json
Saving model checkpoint to output\checkpoint-30000
Configuration saved in output\checkpoint-30000\config.json
Model weights saved in output\checkpoint-30000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-30000\tokenizer_config.json
Special tokens file saved in output\checkpoint-30000\special_tokens_map.json


Training co

In [17]:
#import senteval
#import SentEval.senteval as senteval
#import SentEval_simcse.senteval as senteval
#import SentEval_simcse.senteval.engine as se_engine


def prepare(params, samples):
    return

def batcher(params, batch):
    sentences = [" ".join(s) for s in batch]
    batch = tokenizer.batch_encode_plus(
        sentences,
        return_tensors="pt",
        padding=True,
    )

    for k in batch:
        batch[k] = batch[k].to(device)

    with torch.no_grad():
        outputs = model4(**batch)

    pooled_result = outputs.pooler_output.cpu()

    return pooled_result


def evaluate_model():
    PATH_TO_DATA = "./data"

    params = {"task_path": PATH_TO_DATA, "usepytorch": True, "kfold": 10}
    tasks = ["STSBenchmark", 'STS12', 'STS13', 'STS14', 'STS15']

    se = senteval.engine.SE(params, batcher, prepare)
    #se = se_engine.SE(params, batcher, prepare)
    results = se.eval(tasks)

    print('STS12: ', results["STS12"]["all"]["spearman"]["all"])
    print('STS13: ', results["STS13"]["all"]["spearman"]["all"])
    print('STS14: ', results["STS14"]["all"]["spearman"]["all"])
    print('STS15: ', results["STS15"]["all"]["spearman"]["all"])
    print('STSB: ', results["STSBenchmark"]["test"]["spearman"][0])

    return results

In [18]:
model4.eval()

results4 = evaluate_model()
results4

C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


STS12:  0.6028650839372993
STS13:  0.740729745204717
STS14:  0.692465335997525
STS15:  0.779843190057093
STSB:  0.7226870749431175


{'STSBenchmark': {'train': {'pearson': (0.7654012355643011, 0.0),
   'spearman': SpearmanrResult(correlation=0.7482016388305586, pvalue=0.0),
   'nsamples': 5749},
  'dev': {'pearson': (0.7670311528467801, 5.261345910632041e-291),
   'spearman': SpearmanrResult(correlation=0.7708464868002052, pvalue=1.119990676071557e-295),
   'nsamples': 1500},
  'test': {'pearson': (0.7318742172557201, 1.2511244745112543e-231),
   'spearman': SpearmanrResult(correlation=0.7226870749431175, pvalue=3.856394414212571e-223),
   'nsamples': 1379},
  'all': {'pearson': {'all': 0.7581791623069682,
    'mean': 0.7547688685556003,
    'wmean': 0.7603260289899136},
   'spearman': {'all': 0.7518720772834617,
    'mean': 0.7472450668579604,
    'wmean': 0.7480605503226412}}},
 'STS12': {'MSRpar': {'pearson': (0.6638885051937329, 1.767074037152879e-96),
   'spearman': SpearmanrResult(correlation=0.6412098115217912, pvalue=4.352737200439051e-88),
   'nsamples': 750},
  'MSRvid': {'pearson': (0.7927758724538693, 5.